In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os

%cd "C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal"
from SuportFunctions import *
#print(os.listdir())

warnings.filterwarnings("ignore")

C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal


In [61]:
def leituraApropDiaPAntigo(listaDeDF, formato, abaPlanilha, cabecalho):   
    print('Processo de leitura e limpeza foi iniciado, tempo estimado 1:30 minutos')
    
    for i in range(len(listaDeDF)):
        
        temp_df = pd.read_excel(listaDeDF[i]+formato,sheet_name = abaPlanilha,header = cabecalho)

        agregador_de_data = pd.read_excel(listaDeDF[i]+'.xlsm',sheet_name = 'Apropriação',header = 0)
        
        year = str(agregador_de_data.iat[4, 1])
        month = str(agregador_de_data.iat[4, 3])
        month = month.lower()
        month = month_name(month)
        
        temp_df.columns.values[0] = "Data"
        temp_df['Data'] = temp_df['Data'].astype(str)

        temp_df = temp_df[~temp_df.Data.str.contains("nan")]
        temp_df = temp_df[~temp_df.Data.str.contains("Dia")]
        
        temp_df['Data'] = temp_df['Data'].str[:-2]

        temp_df["Data"] = temp_df["Data"] + "/" + month + "/" + year
        
        temp_df.dropna(subset=['Poço'], inplace=True)
    
        temp_df.columns = temp_df.columns.str.replace(r'\n', r' ',regex = True)
        temp_df.columns = temp_df.columns.str.replace(r'\s+', r' ',regex = True)

        temp_df = renameFormerColumnsApropCN(temp_df)
        temp_df = temp_df.drop(['Medição Operacional - Produção Apropriada ao Poço (m³/dia):','Poços Produtores do Campo'],axis=1)
        
        temp_df = temp_df.loc[:, ~temp_df.columns.str.contains('^Unnamed')]

        temp_df = dataInvalida(temp_df)
                
        temp_df = temp_df.reset_index(drop = True)
        
        dataframes_list.append(temp_df)
        
        print("Leitura concluida - MED FISC "+ month+ "/"+year)
    foo = pd.concat(dataframes_list)

    foo = foo.reset_index(drop = True)
    return foo

def apropDiaNovo(listaDeDF, formato, abaPlanilha, cabecalho):
    for i in range(len(listaDeDF)):
        temp_df = pd.read_excel(listaDeDF[i]+formato,sheet_name = abaPlanilha,header = cabecalho)
        
        #limpeza do cabeçalho
        temp_df.columns = temp_df.columns.str.strip()
        temp_df.columns = temp_df.columns.str.replace(r'\n', r' ',regex = True)
        temp_df.columns = temp_df.columns.str.replace(r'@', r'',regex = True)
        temp_df.columns = temp_df.columns.str.replace(r'\s+', r' ',regex = True)

        #Strip todos os elementos do df
        temp_df = temp_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

        agregador_de_data = pd.read_excel(listaDeDF[i]+'.xlsx',sheet_name = 'Apropriação mensal',header = 0)

        year = str(agregador_de_data.iat[3, 1])
        month = str(agregador_de_data.iat[3, 3])
        month = month.lower()
        month = month_name(month)
        
        temp_df.columns.values[0] = "Data"
        temp_df = temp_df.dropna(subset=['Data'])
        temp_df['Data'] = temp_df['Data'].astype(int)

        temp_df["Data"] = temp_df["Data"].astype(str) + "/" + month + "/" + year

        temp_df = renameFormerColumnsAprop(temp_df)

        temp_df = dataInvalida(temp_df)

        dataframes_list.append(temp_df)
        
        print("Leitura concluida - MED FISC LP " + month+ "/" +year)
        
    foo = pd.concat(dataframes_list)
    foo = foo.reset_index(drop = True)
    
    return foo

def renameFormerColumnsAprop(df):
    df= df.rename(columns={"Potencial de Produção do Poço (m³/dia):":"Potencial de Produção de Óleo","Unnamed: 9":"Potencial de Produção de Gas","Unnamed: 10":"Potencial de Produção de Agua","Potencial de Produção do Poço (m³/dia):.1":"Potencial de Produção de Óleo Corrigido","Unnamed: 12":"Potencial de Produção de Gas Corrigido","Unnamed: 13":"Potencial de Produção de Agua Corrigido","Medição Fiscal - Produção Apropriada ao Poço (m³/dia):":"Medição Fiscal - Produção Apropriada de Óleo","Unnamed: 18":"Medição Fiscal - Produção Apropriada de Gas","Unnamed: 19":"Medição Fiscal - Produção Apropriada de Agua","Tempo de Produção Equivalente (Horas)":"Tempo de Produção Equivalente ÓLEO (Horas)","Tempo de Produção (Horas de Bombeio)":"Tempo de Produção (Horas de Bombeio) ÓLEO"})
    return df

def renameFormerColumnsApropCN(df):
    df= df.rename(columns={"Tempo de Produção (Horas de Bombeio) Condensado":"Tempo de Produção ÓLEO (Horas de Bombeio)","Tempo de Produção Equivalente Condensado (Horas)":"Tempo de Produção Equivalente ÓLEO (Horas)","Potencial de Produção do Poço (m³/dia):":"Potencial de Produção de Óleo","Unnamed: 8":"Potencial de Produção de Gas","Unnamed: 9":"Potencial de Produção de Agua","Potencial de Produção Corrigido do Poço (m³/dia):":"Potencial de Produção de Óleo Corrigido","Unnamed: 11":"Potencial de Produção de Gas Corrigido","Unnamed: 12":"Potencial de Produção de Agua Corrigido","Medição Fiscal - Produção Apropriada ao Poço (m³/dia):":"Medição Fiscal - Produção Apropriada de Óleo","Unnamed: 20":"Medição Fiscal - Produção Apropriada de Gas","Tempo de Produção Equivalente (Horas)":"Tempo de Produção Equivalente ÓLEO (Horas)","Tempo de Produção (Horas de Bombeio)":"Tempo de Produção (Horas de Bombeio) ÓLEO"})
    return df

In [62]:
#Planilhas a serem lidas

Former_month_CARD = ['01_Medição_CARN_Janeiro_2019_(SGM)', '09_Medição_CDN_Outubro_2018_(SGM)_V0', '09_Medição_CDN_Setembro_2018_(SGM)_V10', '11_Medição_CARN_Novembro_2018_(SGM)', '12_Medição_CARN_Dezembro_2018_(SGM)', 'B) 01_Medição_CARN_Fevereiro_2019_(SGM)']

Former_month_CARD_TQ1 = ['01_Medição_CARN_Março_2019_(SGM)(TQ1)', '2.FISCAL_(TQ1) Medição_CARNFEV - Fechado_2020', '2.FISCAL_(TQ1) Medição_CARN_Abril_2019', '2.FISCAL_(TQ1) Medição_CARN_agosto_2019', '2.FISCAL_(TQ1) Medição_CARN_Dezembro_2019', '2.FISCAL_(TQ1) Medição_CARN_Janeiro_2020', '2.FISCAL_(TQ1) Medição_CARN_julho_2019', '2.FISCAL_(TQ1) Medição_CARN_junho_2019', '2.FISCAL_(TQ1) Medição_CARN_Maio_2019', '2.FISCAL_(TQ1) Medição_CARN_novembro_2019', '2.FISCAL_(TQ1) Medição_CARN_outubro_2019', '2.FISCAL_(TQ1) Medição_CARN_setembro_2019']

Former_month_CARD_TQ2 = ['01_Medição_CARN_Março_2019_(SGM)(TQ2)', '2.FISCAL_(TQ2) Medição_CARNFEV - Fechado_2020', '2.FISCAL_(TQ2) Medição_CARN_Abril_2019', '2.FISCAL_(TQ2) Medição_CARN_agosto_2019', '2.FISCAL_(TQ2) Medição_CARN_Dezembro_2019', '2.FISCAL_(TQ2) Medição_CARN_Janeiro_2020', '2.FISCAL_(TQ2) Medição_CARN_julho_2019', '2.FISCAL_(TQ2) Medição_CARN_junho_2019', '2.FISCAL_(TQ2) Medição_CARN_novembro_2019', '2.FISCAL_(TQ2) Medição_CARN_outubro_2019', '2.FISCAL_(TQ2) Medição_CARN_setembro_2019']

New_month_CARD_TQ1 = ['10.1 CARN - Planilha de Medição Fiscal  - OUT - TQ-01_2020', '11.1 CARN - Planilha de Medição Fiscal  - NOV - TQ-01_2020', '4.1-Planilha de Medição Fiscal - CARN - Abril2020- TQ-01 (30 dias)', '5.1- Planilha de Medição Fiscal - CARN - Maio2020- TQ-01 (31dias) - Rateior flare', '6.1 -Planilha de Medição Fiscal - CARN -Junho 2020- TQ-01 (30 dias)', '7.1 CARN - Planilha de Medição Fiscal  - Julho2020- TQ-01', '8.1 CARN - Planilha de Medição Fiscal  - Agosto - TQ-01_2020', '9.1 CARN - Planilha de Medição Fiscal  - Setembro - TQ-01_2020', 'CARN - Planilha de Medição Fiscal  - ABR - TQ-01_2021', 'CARN - Planilha de Medição Fiscal  - FEV - TQ-01_2021', 'CARN - Planilha de Medição Fiscal  - JAN - TQ-01_2021', 'CARN - Planilha de Medição Fiscal  - MAI - TQ-01_2021', 'CARN - Planilha de Medição Fiscal  - MAR - TQ-01_2021', 'CARN - Planilha de Medição Fiscal - DEZ - TQ-01_2020', 'CARN - Planilha de Medição Fiscal - JUL - TQ-01_2021', 'CARN - Planilha de Medição Fiscal - JUN - TQ-01_2021', 'Produção Cardeal - Tanque 01_2020_Mar']

New_month_CARD_TQ2 = ['10.2 CARN - Planilha de Medição Fiscal - OUT - TQ-02_2020', '11.2 CARN - Planilha de Medição Fiscal - NOV - TQ-02_2020', '4.2-Planilha de Medição Fiscal - CARN - Abril2020- TQ-02 (30 dias)', '5.2- Planilha de Medição Fiscal - CARN- Maio2020- TQ-02 (31 dias)', '6.2 -Planilha de Medição Fiscal - CARN - Junho 2020- TQ-02 (30 dias)', '7.2 CARN - Planilha de Medição Fiscal - Julho2020- TQ-02', '8.2 CARN - Planilha de Medição Fiscal - Agosto - TQ-02_2020', '9.2 CARN - Planilha de Medição Fiscal - Setembro - TQ-02_2020', 'CARN - Planilha de Medição Fiscal - ABR - TQ-02_2021', 'CARN - Planilha de Medição Fiscal - DEZ - TQ-02_2020', 'CARN - Planilha de Medição Fiscal - FEV - TQ-02_2021', 'CARN - Planilha de Medição Fiscal - JAN - TQ-02_2021', 'CARN - Planilha de Medição Fiscal - JUL - TQ-02_2021', 'CARN - Planilha de Medição Fiscal - JUN - TQ-02_2021', 'CARN - Planilha de Medição Fiscal - MAI - TQ-02_2021', 'CARN - Planilha de Medição Fiscal - MAR - TQ-02_2021', 'Produção Cardeal - Tanque 02_2020_Mar']

<h2> Planilhas antigas <h2>

In [63]:
%cd "C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Cardeal Nordeste\Antigo"

C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Cardeal Nordeste\Antigo


In [64]:
dataframes_list = []
Aprop_dia_antigo1 = leituraApropDiaPAntigo(Former_month_CARD,'.xlsm', 'ApropDiaP',8)

Aprop_dia_antigo1 = Aprop_dia_antigo1.fillna(0)

for j in range (len(Aprop_dia_antigo1['Data'])):
    if (len(Aprop_dia_antigo1['Data'][j])) == 9:
        Aprop_dia_antigo1['Data'][j] = "0" + Aprop_dia_antigo1['Data'][j]

Aprop_dia_antigo1['Data'] = pd.to_datetime(Aprop_dia_antigo1['Data'],format = '%d/%m/%Y')

Aprop_dia_antigo1['Poço'] = Aprop_dia_antigo1['Poço'].str.replace(r'[\s+]?\(.+\)', r'',regex = True)

Aprop_dia_antigo1 = Aprop_dia_antigo1.sort_values(by = "Data")

Aprop_dia_antigo1['Tanque'] = '01'

Aprop_dia_antigo1

Processo de leitura e limpeza foi iniciado, tempo estimado 1:30 minutos
Leitura concluida - MED FISC 01/2019
Leitura concluida - MED FISC 10/2018
Leitura concluida - MED FISC 09/2018
Leitura concluida - MED FISC 11/2018
Leitura concluida - MED FISC 12/2018
Leitura concluida - MED FISC 02/2019


,Data,Poço,Regime de Produção (Horas Produtoras),Tempo de Produção ÓLEO (Horas de Bombeio),Tempo de Produção (Anular aberto) GÁS,Tempo de Produção Equivalente GÁS (Horas),Tempo de Produção Equivalente ÓLEO (Horas),Potencial de Produção de Óleo,Potencial de Produção de Gas,Potencial de Produção de Agua,Potencial de Produção de Óleo Corrigido,Potencial de Produção de Gas Corrigido,Potencial de Produção de Agua Corrigido,Medição Fiscal - Produção Apropriada de Óleo,Medição Fiscal - Produção Apropriada de Gas,Observações,Falta de energia no campo (min),Tanque
1174,2018-09-01,7GC03RN,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.000,"0,000","0,000","0,000","0,000","0,000",Poço desativado.,0.0,01
1294,2018-09-01,7GC07RN,24.0,0.0,0.0,0.0,0.0,1.277,1100.05,0.127,0,0,0,"0,000","0,000",0,0.0,01
1264,2018-09-01,7GC06RN,24.0,0.0,0.0,0.0,0.0,0.941,73.48,0.358,0,0,0,"0,000","0,000",0,0.0,01
1204,2018-09-01,7GC04RN,24.0,0.0,0.0,0.0,0.0,1.076,76.94,1.734,0,0,0,"0,000","0,000",0,0.0,01
1534,2018-09-01,7GC15RN,24.0,0.0,0.0,0.0,0.0,3.562,14996.42,0.182,0,0,0,"0,000","0,000",0,0.0,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,2019-02-28,7GC13RN,0.0,0.0,0.0,0.0,0.0,2.797,49.78,2.960,0,0,0,"0,000","0,000",0,0.0,01
3048,2019-02-28,7GC14RN,0.0,0.0,0.0,0.0,0.0,0.157,968.08,2.387,0,0,0,"0,000","0,000",0,0.0,01
2684,2019-02-28,1-IMET-10D-BA,24.0,24.0,24.0,24.0,24.0,0.000,73959.52,0.000,0,73959.52,0,0,75863.091819,0,0.0,01
2908,2019-02-28,7GC09RN,0.0,0.0,0.0,0.0,0.0,3.644,1969.95,0.482,0,0,0,"0,000","0,000",0,0.0,01


In [65]:
%cd "C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Cardeal Nordeste\Antigo\Tanque 01"

C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Cardeal Nordeste\Antigo\Tanque 01


In [66]:
dataframes_list = []
Aprop_dia_antigo2 = leituraApropDiaPAntigo(Former_month_CARD_TQ1,'.xlsm', 'ApropDiaP',8)

Aprop_dia_antigo2 = Aprop_dia_antigo2.fillna(0)

for j in range (len(Aprop_dia_antigo2['Data'])):
    if (len(Aprop_dia_antigo2['Data'][j])) == 9:
        Aprop_dia_antigo2['Data'][j] = "0" + Aprop_dia_antigo2['Data'][j]

Aprop_dia_antigo2['Data'] = pd.to_datetime(Aprop_dia_antigo2['Data'],format = '%d/%m/%Y')

Aprop_dia_antigo2['Poço'] = Aprop_dia_antigo2['Poço'].str.replace(r'[\s+]?\(.+\)', r'',regex = True)

Aprop_dia_antigo2 = Aprop_dia_antigo2.sort_values(by = "Data")

Aprop_dia_antigo2['Tanque'] = '01'

Aprop_dia_antigo2.tail()

Processo de leitura e limpeza foi iniciado, tempo estimado 1:30 minutos
Leitura concluida - MED FISC 03/2019
Leitura concluida - MED FISC 02/2020
Leitura concluida - MED FISC 04/2019
Leitura concluida - MED FISC 08/2019
Leitura concluida - MED FISC 12/2019
Leitura concluida - MED FISC 01/2020
Leitura concluida - MED FISC 07/2019
Leitura concluida - MED FISC 06/2019
Leitura concluida - MED FISC 05/2019
Leitura concluida - MED FISC 11/2019
Leitura concluida - MED FISC 10/2019
Leitura concluida - MED FISC 09/2019


,Data,Poço,Regime de Produção (Horas Produtoras),Tempo de Produção ÓLEO (Horas de Bombeio),Tempo de Produção (Anular aberto) GÁS,Tempo de Produção Equivalente GÁS (Horas),Tempo de Produção Equivalente ÓLEO (Horas),Potencial de Produção de Óleo,Potencial de Produção de Gas,Potencial de Produção de Agua,Potencial de Produção de Óleo Corrigido,Potencial de Produção de Gas Corrigido,Potencial de Produção de Agua Corrigido,Medição Fiscal - Produção Apropriada de Óleo,Medição Fiscal - Produção Apropriada de Gas,Observações,Falta de energia no campo (min),Tanque
694,2020-02-28,7GC04RN,0.0,0.0,0.0,0.0,0.0,1.076,76.94,1.734,0,0,0,"0,000","0,000",0,0.0,01
890,2020-02-28,7GC11RN,0.0,0.0,0.0,0.0,0.0,2.340,7.35,2.262,0,0,0,"0,000","0,000",0,0.0,01
610,2020-02-28,1-IMET-10D-BA,24.0,0.0,24.0,24.0,0.0,0.000,18525.88,0.000,0,18525.88,0,"0,000",30154.887071,Poço parado,0.0,01
806,2020-02-28,7GC08RN,0.0,0.0,0.0,0.0,0.0,2.316,61.62,1.390,0,0,0,"0,000","0,000",0,0.0,01
778,2020-02-28,7GC07RN,0.0,0.0,0.0,0.0,0.0,1.277,1100.05,0.127,0,0,0,"0,000","0,000",0,0.0,01


In [67]:
%cd "C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Cardeal Nordeste\Antigo\Tanque 02"

C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Cardeal Nordeste\Antigo\Tanque 02


In [68]:
dataframes_list = []
Aprop_dia_antigo3 = leituraApropDiaPAntigo(Former_month_CARD_TQ2,'.xlsm', 'ApropDiaP',8)

Aprop_dia_antigo3 = Aprop_dia_antigo3.fillna(0)

for j in range (len(Aprop_dia_antigo3['Data'])):
    if (len(Aprop_dia_antigo3['Data'][j])) == 9:
        Aprop_dia_antigo3['Data'][j] = "0" + Aprop_dia_antigo3['Data'][j]

Aprop_dia_antigo3['Data'] = pd.to_datetime(Aprop_dia_antigo3['Data'],format = '%d/%m/%Y')

Aprop_dia_antigo3['Poço'] = Aprop_dia_antigo3['Poço'].str.replace(r'[\s+]?\(.+\)', r'',regex = True)

Aprop_dia_antigo3 = Aprop_dia_antigo3.sort_values(by = "Data")

Aprop_dia_antigo3['Tanque'] = '02'

Aprop_dia_antigo3.tail()

Processo de leitura e limpeza foi iniciado, tempo estimado 1:30 minutos
Leitura concluida - MED FISC 03/2019
Leitura concluida - MED FISC 02/2020
Leitura concluida - MED FISC 04/2019
Leitura concluida - MED FISC 08/2019
Leitura concluida - MED FISC 12/2019
Leitura concluida - MED FISC 01/2020
Leitura concluida - MED FISC 07/2019
Leitura concluida - MED FISC 06/2019
Leitura concluida - MED FISC 11/2019
Leitura concluida - MED FISC 10/2019
Leitura concluida - MED FISC 09/2019


,Data,Poço,Regime de Produção (Horas Produtoras),Tempo de Produção ÓLEO (Horas de Bombeio),Tempo de Produção (Anular aberto) GÁS,Tempo de Produção Equivalente GÁS (Horas),Tempo de Produção Equivalente ÓLEO (Horas),Potencial de Produção de Óleo,Potencial de Produção de Gas,Potencial de Produção de Agua,Potencial de Produção de Óleo Corrigido,Potencial de Produção de Gas Corrigido,Potencial de Produção de Agua Corrigido,Medição Fiscal - Produção Apropriada de Óleo,Medição Fiscal - Produção Apropriada de Gas,Observações,Falta de energia no campo (min),Tanque
666,2020-02-28,7GC03RN,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,"0,000","0,000","0,000","0,000","0,000",Poço desativado.,0.0,02
638,2020-02-28,7GC02RN,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,"0,000","0,000","0,000","0,000","0,000","Poço convertido para injetor, aguardando equip...",0.0,02
610,2020-02-28,1-IMET-10D-BA,24.0,0.0,0.0,24.0,0.0,0.00000,18525.88000,0.00000,0,18525.88,0,"0,000",0,Poço não produz condensado,0.0,02
918,2020-02-28,7GC12RN,0.0,0.0,0.0,0.0,0.0,1.57682,6.25155,0.92438,0,0,0,"0,000","0,000",0,0.0,02
778,2020-02-28,7GC07RN,0.0,0.0,0.0,0.0,0.0,1.27700,1100.05000,0.12700,0,0,0,"0,000","0,000",0,0.0,02


In [69]:
Aprop_dia_antigo = (pd.concat([Aprop_dia_antigo1, Aprop_dia_antigo2, Aprop_dia_antigo3])).reset_index(drop = True)
Aprop_dia_antigo

,Data,Poço,Regime de Produção (Horas Produtoras),Tempo de Produção ÓLEO (Horas de Bombeio),Tempo de Produção (Anular aberto) GÁS,Tempo de Produção Equivalente GÁS (Horas),Tempo de Produção Equivalente ÓLEO (Horas),Potencial de Produção de Óleo,Potencial de Produção de Gas,Potencial de Produção de Agua,Potencial de Produção de Óleo Corrigido,Potencial de Produção de Gas Corrigido,Potencial de Produção de Agua Corrigido,Medição Fiscal - Produção Apropriada de Óleo,Medição Fiscal - Produção Apropriada de Gas,Observações,Falta de energia no campo (min),Tanque
0,2018-09-01,7GC03RN,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.000,"0,000","0,000","0,000","0,000","0,000",Poço desativado.,0.0,01
1,2018-09-01,7GC07RN,24.0,0.0,0.0,0.0,0.0,1.277,1100.05,0.127,0,0,0,"0,000","0,000",0,0.0,01
2,2018-09-01,7GC06RN,24.0,0.0,0.0,0.0,0.0,0.941,73.48,0.358,0,0,0,"0,000","0,000",0,0.0,01
3,2018-09-01,7GC04RN,24.0,0.0,0.0,0.0,0.0,1.076,76.94,1.734,0,0,0,"0,000","0,000",0,0.0,01
4,2018-09-01,7GC15RN,24.0,0.0,0.0,0.0,0.0,3.562,14996.42,0.182,0,0,0,"0,000","0,000",0,0.0,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15482,2020-02-28,7GC04RN,0.0,0.0,0.0,0.0,0.0,1.076,76.94,1.734,0,0,0,"0,000","0,000",0,0.0,01
15483,2020-02-28,7GC11RN,0.0,0.0,0.0,0.0,0.0,2.340,7.35,2.262,0,0,0,"0,000","0,000",0,0.0,01
15484,2020-02-28,1-IMET-10D-BA,24.0,0.0,24.0,24.0,0.0,0.000,18525.88,0.000,0,18525.88,0,"0,000",30154.887071,Poço parado,0.0,01
15485,2020-02-28,7GC08RN,0.0,0.0,0.0,0.0,0.0,2.316,61.62,1.390,0,0,0,"0,000","0,000",0,0.0,01


<h2> Planilhas Novas <h2>

In [70]:
%cd "C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Cardeal Nordeste\Novo\Tanque 01"

C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Cardeal Nordeste\Novo\Tanque 01


In [71]:
dataframes_list = []
ApropriacaoDiaria1 = apropDiaNovo(New_month_CARD_TQ1,'.xlsx','Apropriação diária',3)

ApropriacaoDiaria1 = ApropriacaoDiaria1.fillna(0)

for j in range (len(ApropriacaoDiaria1['Data'])):
    if (len(ApropriacaoDiaria1['Data'][j])) == 9:
        ApropriacaoDiaria1['Data'][j] = "0" + ApropriacaoDiaria1['Data'][j]

ApropriacaoDiaria1['Data'] = pd.to_datetime(ApropriacaoDiaria1['Data'],format = '%d/%m/%Y')

ApropriacaoDiaria1 = ApropriacaoDiaria1.sort_values(by = 'Data')

ApropriacaoDiaria1['Poço'] = ApropriacaoDiaria1['Poço'].str.replace(r'[\s+]?\(.+\)', r'',regex = True)

ApropriacaoDiaria1 = ApropriacaoDiaria1.loc[:, ~ApropriacaoDiaria1.columns.str.contains('^Unnamed')]

ApropriacaoDiaria1['Tanque'] = '01'

ApropriacaoDiaria1

Leitura concluida - MED FISC LP 10/2020
Leitura concluida - MED FISC LP 11/2020
Leitura concluida - MED FISC LP 04/2020
Leitura concluida - MED FISC LP 05/2020
Leitura concluida - MED FISC LP 06/2020
Leitura concluida - MED FISC LP 07/2020
Leitura concluida - MED FISC LP 08/2020
Leitura concluida - MED FISC LP 09/2020
Leitura concluida - MED FISC LP 04/2021
Leitura concluida - MED FISC LP 02/2021
Leitura concluida - MED FISC LP 01/2021
Leitura concluida - MED FISC LP 05/2021
Leitura concluida - MED FISC LP 03/2021
Leitura concluida - MED FISC LP 12/2020
Leitura concluida - MED FISC LP 07/2021
Leitura concluida - MED FISC LP 06/2021
Leitura concluida - MED FISC LP 03/2020


,Data,Poço,Regime de Produção (Horas Produtoras),Tempo de Produção (Horas de Bombeio) Óleo,Tempo de Produção (Anular aberto) GÁS,Tempo de Produção Equivalente GÁS (Horas),Tempo de Produção Equivalente ÓLEO (Horas),Tempo de Produção Equivalente ÁGUA (Horas),Potencial de Produção de Óleo,Potencial de Produção de Gas,Potencial de Produção de Agua,Potencial de Produção de Óleo Corrigido,Potencial de Produção de Gas Corrigido,Potencial de Produção de Agua Corrigido,Medição Fiscal - Produção Apropriada de Óleo,Medição Fiscal - Produção Apropriada de Gas,Medição Fiscal - Produção Apropriada de Agua,Observações gerais dos poços,Tanque
3471,2020-03-01,7-CARN-1D-BA,24.0,0.0,0.0,0.0,0.0,0.0,1.20,45000.00,1.000,0.00,0.00,0.000,0.000000,0.000000,0.000,0,01
3502,2020-03-01,1-IMET-26A-BA,24.0,0.0,0.0,0.0,0.0,0.0,1.80,60000.00,2.500,0.00,0.00,0.000,0.000000,0.000000,0.000,0,01
3564,2020-03-01,1-ALV-11-BA,24.0,0.0,0.0,0.0,0.0,0.0,1.02,50000.00,0.040,0.00,0.00,0.000,0.000000,0.000000,0.000,0,01
3595,2020-03-01,7-CBE-1D-BA,24.0,0.0,0.0,0.0,0.0,0.0,5.00,12000.00,0.050,0.00,0.00,0.000,0.000000,0.000000,0.000,0,01
3533,2020-03-01,1-ALV-7D-BA,24.0,0.0,0.0,0.0,0.0,0.0,1.75,10000.00,0.132,0.00,0.00,0.000,0.000000,0.000000,0.000,0,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,2021-07-31,1-ALV-7D-BA,24.0,24.0,24.0,24.0,24.0,24.0,6.63,157545.26,0.074,6.63,157545.26,0.074,3.358107,171847.683999,0.074,0,01
3043,2021-07-31,1-IMET-10D-BA,24.0,24.0,24.0,24.0,24.0,24.0,0.00,77716.46,0.000,0.00,77716.46,0.000,0.000000,84771.789768,0.000,0,01
3105,2021-07-31,1-IMET-26A-BA,24.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000,0.00,0.00,0.000,0.000000,0.000000,0.000,0,01
3074,2021-07-31,7-CARN-1D-BA,24.0,24.0,24.0,24.0,24.0,24.0,1.89,43386.62,0.430,1.89,43386.62,0.430,0.957288,47325.385502,0.430,0,01


In [72]:
%cd "C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Cardeal Nordeste\Novo\Tanque 02"

C:\Users\geologia\Desktop\Gabriel S.S\Planilha Fiscal\Cardeal Nordeste\Novo\Tanque 02


In [73]:
dataframes_list = []
ApropriacaoDiaria2 = apropDiaNovo(New_month_CARD_TQ2,'.xlsx','Apropriação diária',3)

ApropriacaoDiaria2 = ApropriacaoDiaria2.fillna(0)

for j in range (len(ApropriacaoDiaria2['Data'])):
    if (len(ApropriacaoDiaria2['Data'][j])) == 9:
        ApropriacaoDiaria2['Data'][j] = "0" + ApropriacaoDiaria2['Data'][j]

ApropriacaoDiaria2['Data'] = pd.to_datetime(ApropriacaoDiaria2['Data'],format = '%d/%m/%Y')

ApropriacaoDiaria2 = ApropriacaoDiaria2.sort_values(by = 'Data')

ApropriacaoDiaria2['Poço'] = ApropriacaoDiaria2['Poço'].str.replace(r'[\s+]?\(.+\)', r'',regex = True)

ApropriacaoDiaria2 = ApropriacaoDiaria2.loc[:, ~ApropriacaoDiaria2.columns.str.contains('^Unnamed')]

ApropriacaoDiaria2['Tanque'] = '02'

ApropriacaoDiaria2

Leitura concluida - MED FISC LP 10/2020
Leitura concluida - MED FISC LP 11/2020
Leitura concluida - MED FISC LP 04/2020
Leitura concluida - MED FISC LP 05/2020
Leitura concluida - MED FISC LP 06/2020
Leitura concluida - MED FISC LP 07/2020
Leitura concluida - MED FISC LP 08/2020
Leitura concluida - MED FISC LP 09/2020
Leitura concluida - MED FISC LP 04/2021
Leitura concluida - MED FISC LP 12/2020
Leitura concluida - MED FISC LP 02/2021
Leitura concluida - MED FISC LP 01/2021
Leitura concluida - MED FISC LP 07/2021
Leitura concluida - MED FISC LP 06/2021
Leitura concluida - MED FISC LP 05/2021
Leitura concluida - MED FISC LP 03/2021
Leitura concluida - MED FISC LP 03/2020


,Data,Poço,Regime de Produção (Horas Produtoras),Tempo de Produção (Horas de Bombeio) Óleo,Tempo de Produção (Anular aberto) GÁS,Tempo de Produção Equivalente GÁS (Horas),Tempo de Produção Equivalente ÓLEO (Horas),Tempo de Produção Equivalente ÁGUA (Horas),Potencial de Produção de Óleo,Potencial de Produção de Gas,Potencial de Produção de Agua,Potencial de Produção de Óleo Corrigido,Potencial de Produção de Gas Corrigido,Potencial de Produção de Agua Corrigido,Medição Fiscal - Produção Apropriada de Óleo,Medição Fiscal - Produção Apropriada de Gas,Medição Fiscal - Produção Apropriada de Agua,Observações gerais dos poços,Tanque
3471,2020-03-01,7-CARN-1D-BA,24.0,0.0,0.0,0.0,0.0,0.0,1.20,45000.00,1.000,0.00,0.00,0.000,0.000000,0,0.000,0,02
3502,2020-03-01,1-IMET-26A-BA,24.0,0.0,0.0,0.0,0.0,0.0,1.80,60000.00,2.500,0.00,0.00,0.000,0.000000,0,0.000,0,02
3564,2020-03-01,1-ALV-11-BA,24.0,0.0,0.0,0.0,0.0,0.0,1.02,50000.00,0.040,0.00,0.00,0.000,0.000000,0,0.000,0,02
3595,2020-03-01,7-CBE-1D-BA,24.0,0.0,0.0,0.0,0.0,0.0,5.00,12000.00,0.050,0.00,0.00,0.000,0.000000,0,0.000,0,02
3533,2020-03-01,1-ALV-7D-BA,24.0,0.0,0.0,0.0,0.0,0.0,1.75,10000.00,0.132,0.00,0.00,0.000,0.000000,0,0.000,0,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2702,2021-07-31,1-ALV-7D-BA,24.0,24.0,24.0,24.0,24.0,24.0,6.63,157545.26,0.074,6.63,157545.26,0.074,0.016356,0,0.074,0,02
2640,2021-07-31,7-CARN-1D-BA,24.0,24.0,24.0,24.0,24.0,24.0,1.89,43386.62,0.430,1.89,43386.62,0.430,0.004663,0,0.430,0,02
2764,2021-07-31,7-CBE-1D-BA,24.0,24.0,24.0,24.0,24.0,24.0,5.62,114291.29,0.207,5.62,114291.29,0.207,0.013865,0,0.207,0,02
2733,2021-07-31,1-ALV-11-BA,24.0,24.0,24.0,24.0,24.0,24.0,0.88,24693.75,1.070,0.88,24693.75,1.070,0.002171,0,1.070,0,02


In [89]:
ApropriacaoDiaria = (pd.concat([ApropriacaoDiaria1, ApropriacaoDiaria2])).reset_index(drop = True)

ApropDiaP = (pd.concat([Aprop_dia_antigo, ApropriacaoDiaria])).reset_index(drop = True)

ApropDiaP['Data'] = pd.to_datetime(ApropDiaP['Data'],format = '%Y-%m-%d').dt.strftime("%d-%m-%Y")

pocosInvalidos = ['7GC01RN','7GC02RN','7GC02RN ','7GC03RN','7GC03RN ','7GC04RN','7GC05RN','7GC06RN','7GC07RN','7GC08RN','7GC09RN','7GC10RN','7GC11RN','7GC12RN','7GC13RN','7GC14RN','7GC15RN','1UTC5ARN']

ApropDiaP = ApropDiaP[ApropDiaP.Poço.isin(pocosInvalidos) == False]

ApropDiaP.to_excel("Apropriação diaria CARDND.xlsx")

In [90]:
ApropDiaP['Poço'].value_counts()

1-IMET-10D-BA    1947
1-IMET-3-BA      1947
7-CARN-1D-BA     1036
1-ALV-11-BA      1036
1-ALV-7D-BA      1036
1-IMET-26A-BA    1036
7-CBE-1D-BA      1036
Name: Poço, dtype: int64